In [1]:
from nltk.tokenize import word_tokenize

In [2]:
from konlpy.tag import Kkma

In [3]:
sentence = "오늘 미세먼지는 어제 미세먼지보다 나빠요"

print("원본 : ", sentence )

원본 :  오늘 미세먼지는 어제 미세먼지보다 나빠요


In [4]:
ma = Kkma()

print([token[0] for token in ma.pos(sentence) if token[1].startswith("NN")])

print(ma.nouns(sentence))

['오늘', '미세', '먼지', '미세', '먼지']
['오늘', '미세', '미세먼지', '먼지']


In [5]:
def ngramUmjeol(sentence, n=2):
    result = []
    tokens = list(sentence)
    
    for i in range(len(tokens)-n+1):
#         result.append(tokens[i:i+n])
#         result.append(tuple(tokens[i:i+n]))
        result.append("".join(tokens[i:i+n]))
        
    return result 

In [6]:
def ngramEojeol(sentence, n=2):
    result = []
    tokens = sentence.split()
    
    for i in range(len(tokens)-n+1):
#         result.append(tokens[i:i+n])
#         result.append(tuple(tokens[i:i+n]))
        result.append("".join(tokens[i:i+n]))
        
    return result

In [ ]:
lexicon = list()

th =1 
# Tokenizing
lexicon = [token for token in word_tokenize(sentence) if len(token) > th]
print("어절 단위 분리: ", lexicon)

# 품사
for token in [token for token in word_tokenize(sentence) if len(token) > th]:
    lexicon.extend([token[0] for token in ma.pos(token)])
print("형태소 분석: ", list(set(lexicon)))

# 
for token in [token for token in word_tokenize(sentence) if len(token) > th]:
    lexicon.extend(ma.nouns(token))
print("명사분석: ", list(set(lexicon)))

lexicon.extend(ngramEojeol(" ".join([token[0] for token in  ma.pos(sentence)])))
print("어절(바이그램) 분석: ", list(set(lexicon)))


newLexicon = list()
for term in lexicon:
    lexicon.extend(ngramUmjeol(term))
lexicon.extend([term for term in newLexicon if len(term.strip()) > th])    
print("음절(바이그램) 분석: ", list(set(lexicon)))

    
lexicon

어절 단위 분리:  ['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요']
형태소 분석:  ['미세먼지보다', '어제', '아요', '나빠요', '미세', '보다', '미세먼지는', '먼지', '는', '나쁘', '오늘']
명사분석:  ['미세먼지보다', '어제', '아요', '나빠요', '미세', '보다', '미세먼지는', '미세먼지', '먼지', '는', '나쁘', '오늘']
어절(바이그램) 분석:  ['나빠요', '미세', '먼지는', '미세먼지는', '어제', '아요', '어제미세', '먼지', '보다나쁘', '나쁘', '미세먼지보다', '는어제', '먼지보다', '는', '오늘', '나쁘아요', '보다', '미세먼지', '오늘미세']


In [7]:
from os import listdir

path = "../data"
listdir(path)

# fileids() → 말뭉치 목록을 리턴

def getFileList(base="./", ext="txt"):
    fileList = list()
    
    for file in listdir(base):
        if file.split(".")[-1] == ext:
            fileList.append("{0}/{1}".format(base, file))
            
            
    return fileList

In [8]:
len(getFileList("../data"))

121

In [9]:
def getContent(file):
    with open(file, encoding="utf-8") as f:
        content = f.read()
    return content

In [12]:
# getContent(getFileList("../data")[0])
len(getContent(getFileList("../data")[0]))

354316

In [13]:
content = getContent(getFileList("../data")[0])

In [21]:
import re
from string import punctuation

print(punctuation)

# pattern = re.compile(r"[\/\(\)\]{\}{2,}")
pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation))
pattern.findall(content)
pattern.sub(" ", content)

pattern = re.compile(r"[A-Za-z]{8,}")
pattern.findall(content)

# email #email.com
pattern = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_\.]{3,}(.[A-Za-z0-9\-\_\.]{2,})+)")
pattern.findall(content)
pattern.sub(" ", content)

# www .domain .com
# www .domain .co .kr
# m.domain
pattern = re.compile(r"([A-Za-z0-9\-\_\.]{1,}(.[A-Za-z0-9\-\_\.]{2,})+)")
pattern.findall(content)
content = pattern.sub(" ", content)

pattern = re.compile(r"\s{2,}")
pattern.findall(content)
content = pattern.sub(" ", content)

pattern = re.compile(r"([^ㄱ-ㅎ ㅏ_|가-힣0-9]+)")
pattern.findall(content)
content = pattern.sub(" ", content)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [22]:
def  getPatternList():
    patternList = dict()
    
    pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation))
    patternList['Punctuation'] = pattern
    
    pattern = re.compile(r"[A-Za-z]{8,}")
    patternList['Nonword'] = pattern
    
    pattern = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_\.]{3,}(.[A-Za-z0-9\-\_\.]{2,})+)")
    patternList['Email'] = pattern
    
    
    pattern = re.compile(r"([A-Za-z0-9\-\_\.]{1,}(.[A-Za-z0-9\-\_\.]{2,})+)")    
    patternList['Domain'] = pattern
    
    pattern = re.compile(r"\s{2,}")    
    patternList['Whitespace'] = pattern
    
    pattern = re.compile(r"([^ㄱ-ㅎ ㅏ_|가-힣0-9]+)")
    patternList['Korean'] = pattern

    return patternList

In [24]:
patternList = getPatternList()

content = getContent(getFileList("../data")[0])

for _ in ["Email", "Domain", "Nonword", "Punctuation", "Whitespace"]:
    content = patternList[_].sub(" ", content)

In [ ]:
from nltk.tokenize import sent_tokenize

termList = list() # 지난 시간의  Lexicon
posList = list()
nounLIst = list()
ngramList = list()

th = 1

for sentence in sent_tokenize(content):
    for token in word_tokenize(sentence):
        if len(token) > th:
            termList.append(token)
            posList.append([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th])
            nounLIst.extend([noun for noun in ma.nouns(token) if len(noun) > th])
            ngramList.extend(ngramUmjeol(token))

termLIst = list(set(termList))
posList = list(set(posList))
nounLIst = list(set(nounList))
ngramList = list(set(ngramList))

In [ ]:
len(termList), len(posList), len(nounList), len(ngramList)